In [106]:
#import the requirement
import duckdb
import glob
import pandas as pd

In [3]:
#specify the path to the file
path_data_source = 'C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\data\\'

In [4]:
# Connect to an in-memory DuckDB database
conn = duckdb.connect()

In [98]:
#load the all JSON Files
conn.execute("""
SELECT 
unnest(restaurants, recursive := true)
FROM read_json_auto('C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\data\\*.json')
where 
 results_found is not null
""").fetchdf()

,has_online_delivery,photos_url,url,price_range,apikey,rating_text,rating_color,votes,aggregate_rating,res_id,...,featured_image,currency,id,thumb,establishment_types,events_url,book_url,zomato_events,order_deeplink,order_url
0,0,https://www.zomato.com/orlando/the-coop-winter...,https://www.zomato.com/orlando/the-coop-winter...,2,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Good,9ACD32,"""432""","""3.6""",17066603,...,https://b.zmtcdn.com/data/pictures/3/17066603/...,$,17066603,https://b.zmtcdn.com/data/pictures/3/17066603/...,[],https://www.zomato.com/orlando/the-coop-winter...,None,NaN,None,None
1,0,https://www.zomato.com/orlando/maggianos-littl...,https://www.zomato.com/orlando/maggianos-littl...,4,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Very Good,5BA829,"""886""","""4.4""",17059541,...,https://b.zmtcdn.com/data/pictures/1/17059541/...,$,17059541,https://b.zmtcdn.com/data/pictures/1/17059541/...,[],https://www.zomato.com/orlando/maggianos-littl...,None,NaN,None,None
2,0,https://www.zomato.com/orlando/tako-cheena-by-...,https://www.zomato.com/orlando/tako-cheena-by-...,1,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Very Good,5BA829,"""570""","""4.4""",17064405,...,https://b.zmtcdn.com/data/pictures/5/17064405/...,$,17064405,https://b.zmtcdn.com/data/pictures/5/17064405/...,[],https://www.zomato.com/orlando/tako-cheena-by-...,None,NaN,None,None
3,0,https://www.zomato.com/orlando/bosphorous-turk...,https://www.zomato.com/orlando/bosphorous-turk...,3,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Very Good,5BA829,"""568""","""4.2""",17057797,...,https://b.zmtcdn.com/data/pictures/7/17057797/...,$,17057797,https://b.zmtcdn.com/data/pictures/7/17057797/...,[],https://www.zomato.com/orlando/bosphorous-turk...,None,NaN,None,None
4,0,https://www.zomato.com/orlando/bahama-breeze-i...,https://www.zomato.com/orlando/bahama-breeze-i...,3,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Very Good,5BA829,"""910""","""4.3""",17057591,...,https://b.zmtcdn.com/data/pictures/1/17057591/...,$,17057591,https://b.zmtcdn.com/data/pictures/1/17057591/...,[],https://www.zomato.com/orlando/bahama-breeze-i...,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30168,0,https://www.zomato.com/pocatello-id/senor-igua...,https://www.zomato.com/pocatello-id/senor-igua...,1,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Good,9ACD32,"""108""","""3.6""",17582627,...,,$,17582627,,[],https://www.zomato.com/pocatello-id/senor-igua...,None,NaN,None,None
30169,0,https://www.zomato.com/pocatello-id/sandpiper-...,https://www.zomato.com/pocatello-id/sandpiper-...,3,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Good,9ACD32,"""85""","""3.6""",17582625,...,,$,17582625,,[],https://www.zomato.com/pocatello-id/sandpiper-...,None,NaN,None,None
30170,0,https://www.zomato.com/pocatello-id/texas-road...,https://www.zomato.com/pocatello-id/texas-road...,3,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Good,9ACD32,"""83""","""3.5""",17582668,...,https://b.zmtcdn.com/data/pictures/chains/8/17...,$,17582668,https://b.zmtcdn.com/data/pictures/chains/8/17...,[],https://www.zomato.com/pocatello-id/texas-road...,None,NaN,None,None
30171,0,https://www.zomato.com/pocatello-id/riverwalk-...,https://www.zomato.com/pocatello-id/riverwalk-...,1,b90e6a8c-7384-1031-5a20-c449fe2eb1b1,Good,9ACD32,"""91""","""3.6""",17582498,...,,$,17582498,,[],https://www.zomato.com/pocatello-id/riverwalk-...,None,NaN,None,None


In [101]:
#load the CSV Files
conn.execute("""
SELECT 
*
FROM read_csv('C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\data\\*.csv', ignore_errors=true)
""").fetchdf()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),True,False,False,False,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),True,False,False,False,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),True,False,False,False,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),False,False,False,False,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),True,False,False,False,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9300,5800710,The Manhattan Fish Market,191,Colombo,"31, Deal Place, Off R.A. De Mel Mawatha, Kollu...","Kollupitiya, Colombo 03","Kollupitiya, Colombo 03, Colombo",79.853364,6.906814,"Seafood, Italian",...,Sri Lankan Rupee(LKR),False,False,False,False,4,4.0,Green,Very Good,196
9301,5800433,The Sizzle,191,Colombo,"32, Walukarama Road, Kollupitiya, Colombo 03","Kollupitiya, Colombo 03","Kollupitiya, Colombo 03, Colombo",79.852928,6.904581,"American, Fast Food, Steak, Beverages",...,Sri Lankan Rupee(LKR),False,False,False,False,3,4.2,Green,Very Good,286
9302,5800176,Ministry of Crab,191,Colombo,"Old Colombo Dutch Hospital, Fort, Colombo 01","Old Dutch Hospital, Fort","Old Dutch Hospital, Fort, Colombo",79.844239,6.932814,Seafood,...,Sri Lankan Rupee(LKR),False,False,False,False,4,4.9,Dark Green,Excellent,203
9303,5801321,Cafe Beverly,191,Colombo,"475/C, Sri Jayawardenapura Mawatha, Welikada, ...","Rajagiriya, Colombo","Rajagiriya, Colombo, Colombo",79.902941,6.906815,"Continental, American",...,Sri Lankan Rupee(LKR),False,False,False,False,3,4.1,Green,Very Good,58


In [108]:
#load the Excel with Pandas magically we can call the pandas as table on DuckDB
country = pd.read_excel("C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\data\\Country-Code.xlsx")


In [109]:
country

,Country Code,Country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia
5,148,New Zealand
6,162,Phillipines
7,166,Qatar
8,184,Singapore
9,189,South Africa


In [ ]:
# Close the connection
con.close()